In [1]:
import MyKeys
from openai import OpenAI
client = OpenAI(api_key = MyKeys.API_KEY)
import csv
from amazoncaptcha import AmazonCaptcha
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [3]:
s = webdriver.ChromeService('C:/Users/rakra/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome(service = s)

driver.get('https://www.amazon.ca/errors/validateCaptcha')
driver.implicitly_wait(10) # Let the user actually see something!
link = driver.find_element(By.XPATH, "//div[@class = 'a-row a-text-center']//img").get_attribute('src')
captcha = AmazonCaptcha.fromlink(link)
captcha_value = AmazonCaptcha.solve(captcha)
captchainput = driver.find_element(By.XPATH, "//input[@id = 'captchacharacters']").send_keys(captcha_value)
driver.find_element(By.XPATH, "//button[@class = 'a-button-text']").click()
driver.find_element(By.XPATH, "//input[@id = 'twotabsearchtextbox']").send_keys('3D Printers')
driver.find_element(By.XPATH, "//input[@id = 'nav-search-submit-button']").click()

In [23]:
links = []
reviews = []
ratings = []
prices = []
texts = []
responses = []


for i in range(3):
    Numnames = len(driver.find_elements(By.XPATH, "//a[@class = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']"))
    for j in range(Numnames):
        printer = driver.find_element(By.XPATH, f"(//a[@class = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'])[{j+1}]")
        link = printer.get_attribute('href')
        driver.get(link)

        try:
            whole = driver.find_element(By.XPATH, "(//span[@class = 'a-price-whole'])[1]").text
        except:
            whole = '100000'
        try:
            decimal = driver.find_element(By.XPATH, "(//span[@class = 'a-price-fraction'])[1]").text
        except:
            decimal = '00'
        try:
            rating = driver.find_element(By.XPATH, "(//span/a/span[@class = 'a-size-base a-color-base'])[1]").text
        except:
            rating = 'N/A'
        try:
            review = driver.find_element(By.XPATH, "(//span/a/span[@id = 'acrCustomerReviewText'])[1]").text
        except:
            review = 'N/A'
        try:
            title = driver.find_element(By.XPATH, "//span[@id = 'productTitle']").text
        except:
            title = 'N/A'
        try:
            description = driver.find_element(By.XPATH, "//div[@id = 'productDescription']").text
        except:
            description = 'N/A'

        links.append(link)
        prices.append(whole + "." + decimal)
        ratings.append(rating)
        reviews.append(review)
        texts.append(title + " " + description)
        driver.back()
    driver.find_element(By.XPATH, "//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']").click()

In [ ]:
names = []
isprinter = []
dimensions = []
types = []
speeds = []

Products = len(links)

for i in range(Products):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant extracting 3D printer data."},
            {"role": "user"  , "content": "Tell me the name of the product (excluding extra details). Then tell me if the description given is a 3D printer (yes or no). Then extract the 3D-Printer build volume/size in the format L x W x H. Then the printer type (FDM, SLA, DLP, SLS, etc.), and the printer speed"},
            {"role": "user"  , "content": "Avoid extracting product/package dimensions, ensure units for build volume are in mm, and printer speed is in mm/s. Convert if necessary, only put the numbers in the output."},
            {"role": "user"  , "content": "Put the output in the form 'Name, Yes or no if its a 3D printer, build volume, printer type, printer speed', put 'N/A' if you can not find the information"},
            {"role": "user"  , "content": texts[i]}
        ]   
        )
    response = completion.choices[0].message.content
    data = response.strip().split(",")
    names.append(data[0])
    isprinter.append(data[1])
    dimensions.append(data[2])
    types.append(data[3])
    speeds.append(data[4])

In [26]:
Dict = {
    "name": names,
    "price": prices,
    "Is it a 3D Printer?": isprinter,
    "dimensions (mm)": dimensions,
    "type": types,
    "speed (mm/s)": speeds,
    "rating": ratings,
    "review": reviews,
    "link": links
}

with open('Amazon_3D_Printers.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames = Dict.keys())
    writer.writeheader()
    for i in range(Products):
        row = {key: Dict[key][i] for key in Dict}
        writer.writerow(row)

In [28]:
Dataframe = pd.read_csv('Amazon_3D_Printers.csv')
Dataframe = Dataframe.drop_duplicates(subset = 'name', keep = 'first')
Dataframe = Dataframe[Dataframe['Is it a 3D Printer?'].str.strip().str.lower() == 'yes']
Dataframe.to_csv('Amazon_3D_Printers_Filtered.csv', index = False)